# read sheet rows

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']


In [ ]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ'
RANGE_NAME = 'opportunities_looker!A1:E'


In [ ]:
"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)



In [ ]:
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()
values = result.get('values', [])


In [ ]:
sheet_result = sheet.get(spreadsheetId=SPREADSHEET_ID).execute()

In [ ]:
sheet_result

In [ ]:
if not values:
    print('No data found.')
else:
    print(values)


# change logs for drive

In [37]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import uuid
import requests
# If modifying these scopes, delete the file token.pickle.
# SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive.file']


In [38]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'gsuite_svc_acc.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)
        

In [39]:
# Call the Drive v3 API
results = service.files().list(
    pageSize=100, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        if item['name'] in ('BDR Meeting KPIs and Opportunity Targets', 'Finance Metrics'):
            print(u'{0} ({1})'.format(item['name'], item['id']))
        

Files:
BDR Meeting KPIs and Opportunity Targets (1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ)
Finance Metrics (1yRrgnEdujAsSjIDQEYIVmAHluvYVNBtw9nucoLJ7o9w)


In [19]:
response = service.changes().getStartPageToken().execute()

In [ ]:
response.get('startPageToken')

In [ ]:
page_token = '747'
while page_token is not None:
    response = service.changes().list(pageToken=page_token,
                                            spaces='drive').execute()
    for change in response.get('changes'):
        # Process change
        print('Change found for file: %s' % change.get('fileId'))
    if 'newStartPageToken' in response:
        # Last page, save this token for the next polling interval
        saved_start_page_token = response.get('newStartPageToken')
    page_token = response.get('nextPageToken')

In [ ]:
response

# change list with pageToken

https://developers.google.com/drive/api/v3/push

In [ ]:
import json

In [74]:
channel_id = str(uuid.uuid4())
service = build('drive', 'v3', credentials=creds)
body = {
    "id": channel_id,
    "type": "web_hook",
    "address": 'https://us-central1-data-team-231118.cloudfunctions.net/gsuite_meta/'
}
response = service.changes().list(pageToken = '766').execute()


In [75]:
json.dumps(response)

'{"kind": "drive#changeList", "newStartPageToken": "773", "changes": [{"kind": "drive#change", "type": "file", "changeType": "file", "time": "2019-10-02T06:02:08.276Z", "removed": false, "fileId": "1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ", "file": {"kind": "drive#file", "id": "1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ", "name": "BDR Meeting KPIs and Opportunity Targets", "mimeType": "application/vnd.google-apps.spreadsheet"}}]}'

In [ ]:
import json
SCOPES = ['https://www.googleapis.com/auth/drive.file']

header = {
    'Authorization': 'Bearer ',
    'Content-Type': 'application/json'
}

body = {
    "id": channel_id,
    "type": "web_hook",
    "address": 'https://us-central1-data-team-231118.cloudfunctions.net/gsuite_meta/'
}

file_id = '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ'
r = requests.post(url=f'https://www.googleapis.com/drive/v3/files/{file_id}/watch', data=json.dumps(body), headers=header) 


In [ ]:
r.json()

# access token?

In [ ]:
import jwt
import json
import uuid
import requests

In [ ]:
with open('gsuite_credentials.json') as f:
    svc_acc_json = json.load(f)


In [ ]:
encoded_jwt = jwt.encode(svc_acc_json, 'secret', algorithm='HS256')

In [ ]:
encoded_jwt

In [29]:
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client.client import OAuth2Credentials

In [54]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(
      'gsuite_svc_acc.json', scopes=['https://www.googleapis.com/auth/drive.file'])
access_token_info = credentials.get_access_token()

# credentials = OAuth2Credentials.from_json(
#       'gsuite_credentials.json')
# credentials.get_access_token()

In [57]:
access_token_info.access_token

'ya29.c.Kl2UB43n3y0ZzAYKwlRgCbbUZ3LdgVuiqzrMLzTZutRllEvU7KBfybCbkM1RdCRzCO2g0h0eATNibrJKPmZwcgq3fTnNeKU_1Jr55jRE8QZ-ehwlmsUUbgWeojeWVhY'

In [68]:
uuid.uuid4()

UUID('c88f39b9-657f-4d31-b3f5-78bee3dc6831')

In [13]:
import uuid
import requests
import json

In [65]:
channel_id = str(uuid.uuid4())
SCOPES = ['https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive.readonly']

token = access_token_info.access_token

header = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

body = {
#     "kind": "api#channel",
    "id": channel_id,
    "type": "web_hook",
    "address": 'https://us-central1-data-team-231118.cloudfunctions.net/gsuite_meta/'
}

file_id = '1yRrgnEdujAsSjIDQEYIVmAHluvYVNBtw9nucoLJ7o9w'
r = requests.post(url=f'https://www.googleapis.com/drive/v3/file/{file_id}/watch?pageToken=766', 
                  data=json.dumps(body), headers=header) 


In [59]:
r.text

'Not Found'

In [66]:
r_list_files = requests.get(url='https://www.googleapis.com/drive/v3/drives', headers=header)

In [67]:
r_list_files.json()

{'error': {'errors': [{'domain': 'global',
    'reason': 'insufficientPermissions',
    'message': 'Insufficient Permission: Request had insufficient authentication scopes.'}],
  'code': 403,
  'message': 'Insufficient Permission: Request had insufficient authentication scopes.'}}